## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saldanha,ZA,-33.0117,17.9442,68.00,73,0,3.44,clear sky
1,1,Hithadhoo,MV,-0.6000,73.0833,83.03,69,89,6.73,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,64.40,68,0,8.05,clear sky
3,3,Bengkulu,ID,-3.8004,102.2655,76.62,90,94,3.91,moderate rain
4,4,Agirish,RU,61.9247,63.0231,27.01,93,11,8.14,few clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp= float(input("What is the maximum temperature you would like for your vacation? "))
min_temp= float(input("What is the minimum temperature you would like for your vacation? "))


What is the maximum temperature you would like for your vacation? 90
What is the minimum temperature you would like for your vacation? 75


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# filter the dataset to find the citites that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,83.03,69,89,6.73,overcast clouds
3,3,Bengkulu,ID,-3.8004,102.2655,76.62,90,94,3.91,moderate rain
6,6,Dingle,PH,10.9995,122.6711,76.06,89,72,6.69,broken clouds
12,12,Thinadhoo,MV,0.5333,72.9333,80.33,75,72,1.95,broken clouds
17,17,Bambous Virieux,MU,-20.3428,57.7575,78.80,89,75,2.30,light rain
20,20,Pizarro,CO,4.9533,-77.3660,76.75,83,100,11.50,overcast clouds
21,21,Bathsheba,BB,13.2167,-59.5167,82.40,54,40,23.02,scattered clouds
22,22,Taoudenni,ML,22.6783,-3.9836,79.70,6,3,11.16,clear sky
23,23,Bandarbeyla,SO,9.4942,50.8122,79.03,72,69,15.59,broken clouds
25,25,Cidreira,BR,-30.1811,-50.2056,76.50,80,98,8.63,moderate rain


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                222
City                   222
Country                222
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,83.03,overcast clouds,-0.6000,73.0833,
3,Bengkulu,ID,76.62,moderate rain,-3.8004,102.2655,
6,Dingle,PH,76.06,broken clouds,10.9995,122.6711,
12,Thinadhoo,MV,80.33,broken clouds,0.5333,72.9333,
17,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,
20,Pizarro,CO,76.75,overcast clouds,4.9533,-77.3660,
21,Bathsheba,BB,82.40,scattered clouds,13.2167,-59.5167,
22,Taoudenni,ML,79.70,clear sky,22.6783,-3.9836,
23,Bandarbeyla,SO,79.03,broken clouds,9.4942,50.8122,
25,Cidreira,BR,76.50,moderate rain,-30.1811,-50.2056,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [40]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,83.03,overcast clouds,-0.6000,73.0833,Scoop Guest House
3,Bengkulu,ID,76.62,moderate rain,-3.8004,102.2655,Grage Hotel Bengkulu
6,Dingle,PH,76.06,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
17,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,Casa Tia Villa
20,Pizarro,CO,76.75,overcast clouds,4.9533,-77.3660,Hotel Limonar
21,Bathsheba,BB,82.40,scattered clouds,13.2167,-59.5167,Atlantis Hotel
23,Bandarbeyla,SO,79.03,broken clouds,9.4942,50.8122,JABIR HOTEL
25,Cidreira,BR,76.50,moderate rain,-30.1811,-50.2056,Hotel Castelo
34,Touros,BR,80.60,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
35,Iberia,US,77.00,mist,29.9502,-91.7507,back to back rv park


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F <dd/>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))